<a href="https://colab.research.google.com/github/mipypf/practical-mi-guide/blob/develop/chapter4/src/llm_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAGによるLLMへの知識付与

## Google colabを使用の場合、ランタイムのタブから「ランタイムのタイプを変更」→ハードウェアアクセラレータと進み、T4 GPUを選択

In [22]:
# 以下のコマンドでGPUが使用可能かを確認
! nvidia-smi

Fri Feb  7 07:09:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P0             30W /   70W |    5774MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Google Colabの準備

In [23]:
# Google Colabを利用する場合はTrue、そうでない場合はFalseとする
colab = True

In [24]:
# Google Colabのファイルをクリックし、MIハンドブック_はじめに.pdfをドラッグ＆ドロップしてアップロード
if colab:
  INPUT_FILE_PATH = "./"
  OUTPUT_FILE_PATH = "./"
else:
  INPUT_FILE_PATH = "../input/"
  OUTPUT_FILE_PATH = "../output/"

## ライブラリをインポート

In [25]:
!pip install faiss-cpu #執筆当時のGoogle Colabではfaiss-gpuのpip installではERRORが発生する
!pip install pypdf==4.1.0
!pip install fugashi #"tohoku-nlp/bert-base-japanese-char"の駆動に必要
!pip install ipadic #"tohoku-nlp/bert-base-japanese-char"の駆動に必要
!pip install unidic_lite #"tohoku-nlp/bert-base-japanese-char"の駆動に必要

In [26]:
import warnings

warnings.filterwarnings("ignore")

import re

import faiss
import numpy as np
import torch
from IPython.display import Markdown
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

## データを読み込み文章を一定のまとまり（チャンク）に分割し、前処理を実施

In [27]:
# PDFを読み込む関数
def load_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text


# テキストをチャンクに分割する関数
def split_text(text, chunk_size=400):
    chunks = [text[i : i + chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks


# PDFから抽出したテキストから改行コードを削除する関数
def clean_text(text):
    return text.replace("\n", "").strip()

In [28]:
# PDFファイルをロードしてチャンクに分割
file_path = INPUT_FILE_PATH + "MIハンドブック_はじめに.pdf"
text = load_pdf(file_path)
cleaned_text = clean_text(text)
documents = split_text(cleaned_text)

In [29]:
documents

['はじめに   良い材料を早くつくりたいので、 マテリアルズ・インフォマティクス（MI）を活用して”いる”。このような 声はもう珍しくありません。   MIブームに端を発して、大学・企業 ともにMIの活用検討 が始められて 既にある程度の期間が経過しました。すでに MIに取り組んでいる 大学・企業 からはMI活用の成果が多く報告されており、 MIは期待に応えられるということが共通理解 になりつつあります。 MIとは？有効なものなのか？が 大勢を占めていた 数年前とは隔世の感があります。   本書を手に取っていただいた 方は、材料開発者 として、あるいは 材料開発を支援するデータ分析者として「 MI を活用して材料開発 を加速したい。 」という気持ちだと考えています。      著者の高原と福岡は多くの民間企業 などに対して MI 活用の支援をさせて頂いています。 学生時代 には二人とも材料工',
 '学 を専攻し、それぞれ 別メーカーでMIを活用した材料開発業務 を経て、現在はコンサルティング・データ 分析・講演・教育・ データ活用システム の提供といった 様々な立場・内容 で活動を行っています。    MIの普及に伴い、様々な書籍やセミナー が発刊・実施 されています。しかしそれは、 MIとは？という 概論か、整形されたデータに対して分析アルゴリズムを適用するものが 主と感じています。   材料開発 の実務の中で MI を活用するには、さらに 踏み込んだ情報が必要です。  例えば、データの整形方法。 MIに限らず機械学習・ データ分析の入門書では分析可能 な形に整形されたデータから分析がスタート します。 一方で、MI が普及していない 材料開発 の現場では、データ分析フレンドリー な形でデータが蓄積されていません。そのため、その データを分析可能 にする手順が分からずMIの活用を断',
 '念してしまうことがあります。   次にデータや分析結果 の見方という観点。本書執筆 の時点では、AIにデータを丸ごと投げ込み、出てきた結果をそのまま 実験すればより 良い材料が出来るということは、ほとんどの 場合ありません。 手持ちのデータのどのような点に着目し、データ分析を適用するか。 出てきた分析結果 をどのように 材料開発の加速に役立てていくか。こういった ノウハウ 

## Transformer系列のモデルを用いたベクトル化を行う

In [30]:
# SentenceTransformerを使用して埋め込みを生成
embedding_model = SentenceTransformer("tohoku-nlp/bert-base-japanese-char")
embeddings = embedding_model.encode(documents, convert_to_tensor=False)

In [31]:
embeddings

array([[-0.03047474, -0.02585311,  0.08110178, ...,  0.11155611,
         0.03575248,  0.07502937],
       [-0.01095174, -0.07489808, -0.0031864 , ...,  0.10563548,
        -0.02885757,  0.12586543],
       [-0.02559055, -0.05745022,  0.17086999, ...,  0.07885666,
        -0.08108613,  0.03250976],
       [ 0.04616659, -0.06611266,  0.08924499, ...,  0.11290875,
         0.02767573, -0.02722615],
       [ 0.03609647, -0.116508  ,  0.07292653, ...,  0.10481993,
        -0.01766475,  0.12581478],
       [ 0.0936287 , -0.02034189, -0.06240779, ...,  0.05990342,
         0.05203226, -0.04525827]], dtype=float32)

## 検索ができるようにベクトル化したテキストデータにインデックスを付与

In [32]:
# FAISSインデックスを作成
d = embeddings[0].shape[0]  # 埋め込みの次元
index = faiss.IndexFlatL2(d)
index.add(np.array(embeddings))

## LLMを読み込む

### HuggingFaceでの作業
 - https://huggingface.co/google/gemma-2-2b-jpn-it にアクセスしてログインし、ライセンス認証を行う。（Singn up未の場合は、登録から行う。）
 - HuggingFaceのトークンをhttps://huggingface.co/settings/tokens から取得
 - Google Colabのシークレット（鍵マークのアイコン）をクリックして開き、「+ 新しいシークレットを追加」からHF_TOKENという名前を付けて、値の欄に取得したトークンを入力
 - ノートブックからのアクセスを有効にする

In [33]:
from google.colab import userdata

from huggingface_hub import login
login(token=userdata.get('HF_TOKEN'))

In [34]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-jpn-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-jpn-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## テキスト生成のための設定

In [35]:
# テキスト生成パイプラインを設定
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [36]:
def ask_question(question):
    # 質問を埋め込みに変換
    question_embedding = embedding_model.encode([question], convert_to_tensor=False)

    # FAISSで最も近いチャンクを検索
    _, indices = index.search(np.array(question_embedding), k=1)
    retrieved_text = documents[indices[0][0]]

    # 質問と関連情報、回答の間に改行を追加
    input_text = f"質問: {question}\n\n関連情報: {retrieved_text}\n\n回答: "

    # パイプラインを使用して回答を生成
    outputs = text_gen_pipeline(input_text, max_new_tokens=256)

    # リストから文字列を取得
    assistant_response = outputs[0]["generated_text"].strip()

    return assistant_response

## RAGを使用せず知識付与を行わない状態でLLMに質問する

In [37]:
question = "MIとは何ですか？"

outputs = text_gen_pipeline(question, max_new_tokens=256)
response = outputs[0]["generated_text"].strip()
print(response)

MIとは何ですか？

「**AI**」の略称で、人工知能の技術です。

**AIとは？**

* **人間のような思考・判断能力を持つコンピュータ**
* **学習・分析・予測・行動**を自動化できる能力を持つ

**AIの活用例**

* **自動運転車**
* **医療診断**
* **顧客対応**
* **商品推薦**
* **翻訳**
* **画像認識**


In [38]:
display(Markdown(response))

MIとは何ですか？

「**AI**」の略称で、人工知能の技術です。

**AIとは？**

* **人間のような思考・判断能力を持つコンピュータ**
* **学習・分析・予測・行動**を自動化できる能力を持つ

**AIの活用例**

* **自動運転車**
* **医療診断**
* **顧客対応**
* **商品推薦**
* **翻訳**
* **画像認識**

## RAGを使用して知識付与を行った状態でLLMに質問する

In [39]:
print(ask_question(question))

質問: MIとは何ですか？

関連情報: はじめに   良い材料を早くつくりたいので、 マテリアルズ・インフォマティクス（MI）を活用して”いる”。このような 声はもう珍しくありません。   MIブームに端を発して、大学・企業 ともにMIの活用検討 が始められて 既にある程度の期間が経過しました。すでに MIに取り組んでいる 大学・企業 からはMI活用の成果が多く報告されており、 MIは期待に応えられるということが共通理解 になりつつあります。 MIとは？有効なものなのか？が 大勢を占めていた 数年前とは隔世の感があります。   本書を手に取っていただいた 方は、材料開発者 として、あるいは 材料開発を支援するデータ分析者として「 MI を活用して材料開発 を加速したい。 」という気持ちだと考えています。      著者の高原と福岡は多くの民間企業 などに対して MI 活用の支援をさせて頂いています。 学生時代 には二人とも材料工

回答: 

MI（Materials Informatics）とは、材料の特性や性能を分析し、設計・開発を効率化するための、材料科学とデータ分析の融合技術です。


**簡単に言うと:**

* 材料のデータを集めて分析し、より良い材料の開発を支援する技術です。
* 材料の特性や性能を数値化し、設計段階で最適な材料を選択できるようになります。


**MIのメリット:**

* **材料開発のスピードアップ:**  大量のデータから、最適な材料を見つけることができます。
* **コスト削減:**  設計段階で失敗を減らし、無駄な材料や工程を削減できます。
* **材料の性能向上:**  データ分析によって、材料の性能を向上させるための設計が可能になります。


In [40]:
display(Markdown(ask_question(question)))

質問: MIとは何ですか？

関連情報: はじめに   良い材料を早くつくりたいので、 マテリアルズ・インフォマティクス（MI）を活用して”いる”。このような 声はもう珍しくありません。   MIブームに端を発して、大学・企業 ともにMIの活用検討 が始められて 既にある程度の期間が経過しました。すでに MIに取り組んでいる 大学・企業 からはMI活用の成果が多く報告されており、 MIは期待に応えられるということが共通理解 になりつつあります。 MIとは？有効なものなのか？が 大勢を占めていた 数年前とは隔世の感があります。   本書を手に取っていただいた 方は、材料開発者 として、あるいは 材料開発を支援するデータ分析者として「 MI を活用して材料開発 を加速したい。 」という気持ちだと考えています。      著者の高原と福岡は多くの民間企業 などに対して MI 活用の支援をさせて頂いています。 学生時代 には二人とも材料工

回答: 

MI（Materials Informatics）とは、材料の特性や性能を分析し、設計・開発を効率化するための、材料科学とデータ分析の融合技術です。


**簡単に言うと:**

* 材料のデータを集めて分析し、より良い材料の開発を支援する技術です。
* 材料の特性や性能を数値化し、設計段階で最適な材料を選択できるようになります。


**MIのメリット:**

* **材料開発のスピードアップ:**  大量のデータから、最適な材料を見つけることができます。
* **コスト削減:**  設計段階で失敗を減らし、無駄な材料や工程を削減できます。
* **材料の性能向上:**  データ分析によって、材料の性能を向上させるための設計が可能になります。

## 実行環境のライブラリverを保存

In [41]:
!pip freeze > requirements_llm_rag.txt

In [42]:
from google.colab import files

files.download('requirements_llm_rag.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>